# Recommender Systems Deep-Dive Lab

There are many different approaches that we can take when creating recommender systems. In the Intro to Recommender Systems lesson and lab, we put together a user similarity based recommender that first calculated the similarities between users and then leveraged a rank-based item recommender within each group of similar customers. In other words, for a given user, our recommender found the top 5 customers who were the most similar to them, aggregated and ranked the purchases of those 5 customers, and then recommended the top 5 most popular products among that group of similar users to the customer.

In this lab, we are going to start out with the same data set, but we are going to dive deeper into the analysis of customers and products and look at an alternative way to generate recommendations.

We will begin by importing everything we will need for this lab (libraries, data set, etc.).

In [23]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [24]:
data = pd.read_csv('../data/customer_product_sales.csv')

## Data Preparation

We will then put together the foundational transformations of the data that we will need to eventually produce recommendations. The steps in this section should be familiar to you, as you would have had to tranform the data in this manner to create the user similarity based recommender in the Intro to Recommender Systems lab.

First, we will create a data frame that contains the total quantity of each product purchased by each customer.

In [25]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()

Then, we want to create a matrix that has customers on one axis, products on the other, and the quantity purchased as the values. There will be many instances where a customer has not purchased a product, which by default will be expressed with a null value. We will want to replace those nulls with zeros by appending `.fillna(0)` to our pivot table.

In [26]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', 
                                                columns='CustomerID', 
                                                index='ProductName', 
                                                aggfunc='sum').fillna(0)

prod_cust_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In the pivot table we created, the rows represent the products and the columns represent the customers. Depending on what need to do with the matrix, we may instead need to transpose it so that the rows represent customers and the columns to represent products. We can do this easily by appending `.T` to our product customer matrix.

In [27]:
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


Another thing we may want to do is normalize the values across rows or columns of the matrix so that all the values are between 0 and 1. Doing this for customers would help us identify customers that may have purcahsed a similar mix of products even though some of those customers may have purchased large quantities while other may have purchased smaller quantities. Doing this for products would help us better identify products that have been purchased by similar groups of customers regardless of the quantities purchased.

We can normalize across rows for each matrix as follows.

In [28]:
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## User Similarity Based Recommendations

The next step in creating recommendations is calculating similarities. For our user similarity based recommender, we calculated them between customers.

In [29]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.854082,0.859145,0.852783,0.856118,0.853239,0.849549,0.848801,0.851224,0.849892,...,0.855130,0.846812,0.850301,0.841254,0.852221,0.846485,0.852848,0.847637,0.856465,0.851614
200,0.854082,1.000000,0.857935,0.855900,0.856445,0.853129,0.863562,0.850521,0.857381,0.856621,...,0.864397,0.856014,0.862447,0.854629,0.856670,0.854046,0.868990,0.857312,0.861508,0.857184
264,0.859145,0.857935,1.000000,0.851801,0.860050,0.851262,0.855847,0.846677,0.854206,0.848997,...,0.861268,0.850766,0.854178,0.848507,0.858909,0.860153,0.859773,0.854246,0.864699,0.853668
356,0.852783,0.855900,0.851801,1.000000,0.852571,0.849431,0.853831,0.851902,0.847607,0.846328,...,0.857171,0.848015,0.852222,0.847706,0.851413,0.851289,0.851219,0.852260,0.860721,0.854377
412,0.856118,0.856445,0.860050,0.852571,1.000000,0.855755,0.858976,0.857221,0.854857,0.855991,...,0.859117,0.862847,0.854943,0.850044,0.858385,0.856629,0.855052,0.856608,0.865269,0.862253


Once we had our similarity matrix, then we could produce recommendations for each user and package all the recommendations into a data frame.

In [30]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])

user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,General Purpose Trigger,Cookie Chocolate Chip With,Chef Hat 20cm,Pasta - Angel Hair
2,264,Ezy Change Mophandle,Eggplant - Asian,Scallops - 10/20,Cinnamon Buns Sticky,Wine - Ej Gallo Sierra Valley
3,356,Tea - Herbal Sweet Dreams,Curry Paste - Madras,Tea - English Breakfast,Juice - Orange,Ecolab - Lime - A - Way 4/4 L
4,412,Cake - Box Window 10x10x2.5,Beef - Montreal Smoked Brisket,Bread - Raisin Walnut Oval,"Mushroom - Trumpet, Dry",Cheese - Mix


## Deeper Dive Into Our User Similarity Recommendations

Let's deconstruct what we've done and take a deeper dive into how we put this together. Doing this will equip us with the knowledge to be able to put together an item-based similarity recommender in the next section. 

After creating an empty dictionary to store our recommendations and getting a unique list of customer IDs to iterate through, we are first identifying the top 5 similar customers to the customer we are trying to generate recommendations for. Let's plug in customer ID 33 and see what results we get.

In [31]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 6001, 79458, 33759]

What we get is a list containing the 5 customer IDs of the customers whose purchase behavior is most similar to customer 33. We then go back to our customer_products data frame and select just the purchases where the customer ID is in our list of similar customers. We aggregate on product name, summing up the total quantity purchased of each product by all 5 similar customers, and then we rank them by sorting in descending order by the total quantity.

In [32]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


We now have a ranked list of products that similar customers have purchased, but we haven't taken into consideration yet whether our target customer already purchases any of those items. We want to recommend them items that they might like but haven't purchased before. So we will merge the list of ranked products with our target customer's purchase list and keep only the records for items that the customer has not purchased. These will be the items that we recommend to the customer.

In [33]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Item Similarity Based Recommendations

In this section, you will create an item similarity based recommender system in a step-by-step fashion. Whereas our user similarity based recommender leveraged similarities between customers, this recommender will utilize similarities between products. You already have all the tools in your toolbox, so follow each of the steps below to complete this lab.

### Step 1: Create a product distance matrix.

In [284]:
prod_dist = pd.DataFrame(1/(1 + squareform(pdist(prod_cust_pivot, 'cosine'))),
                         index=prod_cust_pivot.index, columns=prod_cust_pivot.index)
prod_dist

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,1.000000,0.525261,0.526165,0.555104,0.564050,0.519332,0.536893,0.517640,0.547829,0.575260,...,0.537868,0.550524,0.549926,0.525350,0.517278,0.550767,0.532645,0.534287,0.545650,0.523806
"Appetizer - Mini Egg Roll, Shrimp",0.525261,1.000000,0.527240,0.529561,0.544405,0.520697,0.527755,0.556529,0.551599,0.533705,...,0.547988,0.541984,0.550369,0.525597,0.525008,0.515063,0.537960,0.548444,0.536942,0.538182
Appetizer - Mushroom Tart,0.526165,0.527240,1.000000,0.545139,0.516876,0.528529,0.529274,0.531369,0.542411,0.517489,...,0.539689,0.528103,0.531201,0.552545,0.540265,0.545464,0.540018,0.539135,0.540934,0.526612
Appetizer - Sausage Rolls,0.555104,0.529561,0.545139,1.000000,0.544081,0.550881,0.526215,0.535372,0.526842,0.536206,...,0.538307,0.549571,0.548994,0.565185,0.530477,0.513588,0.536583,0.532383,0.531658,0.527799
Apricots - Dried,0.564050,0.544405,0.516876,0.544081,1.000000,0.538834,0.531543,0.543368,0.532270,0.544630,...,0.534045,0.540305,0.553428,0.550042,0.517577,0.511678,0.547295,0.542470,0.539973,0.550194
Apricots - Halves,0.519332,0.520697,0.528529,0.550881,0.538834,1.000000,0.550772,0.540168,0.518473,0.548798,...,0.543641,0.554083,0.523545,0.524914,0.538305,0.531372,0.529866,0.550401,0.547453,0.538306
Apricots Fresh,0.536893,0.527755,0.529274,0.526215,0.531543,0.550772,1.000000,0.532219,0.558042,0.524654,...,0.537553,0.553167,0.537163,0.515154,0.528246,0.527457,0.551021,0.536259,0.537915,0.525967
Arizona - Green Tea,0.517640,0.556529,0.531369,0.535372,0.543368,0.540168,0.532219,1.000000,0.536807,0.527426,...,0.525212,0.557115,0.534820,0.539196,0.537092,0.534786,0.535536,0.534222,0.548134,0.540596
Artichokes - Jerusalem,0.547829,0.551599,0.542411,0.526842,0.532270,0.518473,0.558042,0.536807,1.000000,0.536133,...,0.542885,0.532388,0.536525,0.543014,0.532853,0.539677,0.532445,0.547585,0.539658,0.542195


### Step 2: Get the products purchased for a specific customer of your choice.

In [285]:
prod = [x for x in prod_cust_pivot[prod_cust_pivot[33] > 0][33].sort_values(ascending=False).index]

### Step 3: For each product the customer purchased, get a list of the top 5 similar products. Package the lists into a nested list, flatten the list, and then filter out any products the customer has already purchased.

In [286]:
prod = [x for x in prod_cust_pivot[prod_cust_pivot[33] > 0][33].sort_values(ascending=False).index]
top_related_product = []
df=[]
for product in prod:
    for i in range(4):
        top_related_product.append(prod_dist[product].sort_values(ascending=False)[1:].head().index[i])

recom = [x for x in top_related_product if not x in prod]

# Step 4: Count the number of times each similar product occurs in your filtered list. Sort and return a list containing the top 5 items.

In [287]:
srecom = set(recom)
sprod = {}
for x in srecom:
    sprod[x] = recom.count(x)
sprod = pd.DataFrame(sprod.items())
sprod = sprod.sort_values(by=[1], ascending=False).head().reset_index().drop(["index",1], axis=1)
df = [[x for x in sprod[0]]]
df = pd.DataFrame(df, columns=["P1","P2","P3","P4","P5"])
display(df)

,P1,P2,P3,P4,P5
0,Beans - Wax,Garbage Bags - Clear,Beef - Montreal Smoked Brisket,Bread Fig And Almond,Turkey - Oven Roast Breast


### Step 5: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 2 through 4 for each and appending the results of each iteration to the dictionary you created.

In [290]:
df=[]
#Lista de productos que ha comprado el cliente
for customer in cust_prod_pivot.head(50).index:
    prod = [x for x in prod_cust_pivot[prod_cust_pivot[customer] > 0][customer].sort_values(ascending=False).head().index]
    #Recomendación de 5 productos similares a los productos comprados
    top_related_product = [prod_dist[product].sort_values(ascending=False)[1:].head().index[i] for i in range(4) for product in prod]
    #Eliminamos los productos que ya ha comprado el cliente
    recom = [x for x in top_related_product if not x in prod]
    #Filtramos los 5 productos más recomendados en función de los productos comprados
    srecom = set(recom)
    sprod = {}
    for x in srecom:
        sprod[x] = recom.count(x)
    sprod = pd.DataFrame(sprod.items())
    sprod = sprod.sort_values(by=[1], ascending=False).head()
    #Agregamos los valores de cada cliente a una lista con todos los clientes
    df.append([x for x in sprod[0]])
    
#Creo un dataframe con los productos recomendados para cada cliente
df = pd.DataFrame(df, columns=["Producto 1","Producto 2","Producto 3","Producto 4","Producto 5"], index=cust_prod_pivot.head(50).index)
display(df)

,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
CustomerID,,,,,
33,Smoked Paprika,"Liners - Banana, Paper",Beans - Wax,Beef - Short Loin,Wine - Hardys Bankside Shiraz
200,"Rosemary - Primerba, Paste",Garbage Bags - Clear,Sugar - Fine,Vanilla Beans,Cheese - Mozzarella
264,Bread - Calabrese Baguette,Dc Hikiage Hira Huba,Browning Caramel Glace,Foam Dinner Plate,Olives - Kalamata
356,Watercress,Turkey - Oven Roast Breast,Wine - Chardonnay South,"Fish - Scallops, Cold Smoked",Rabbit - Whole
412,Sunflower Seed Raw,Artichokes - Jerusalem,Wiberg Super Cure,Olive - Spread Tapenade,"Bread - Roll, Soft White Round"
464,Bread - Italian Roll With Herbs,Rosemary - Dry,Table Cloth 54x72 White,Beef - Short Loin,Dc - Frozen Momji
477,Chocolate - Feathers,Bread - Calabrese Baguette,"Pepper - Paprika, Hungarian",Bread Crumbs - Japanese Style,Olive - Spread Tapenade
639,"Rum - Coconut, Malibu",Turkey - Oven Roast Breast,Longos - Grilled Salmon With Bbq,Cod - Black Whole Fillet,"Wine - White, Schroder And Schyl"
649,Nantucket - Pomegranate Pear,Bread - Calabrese Baguette,Bread Crumbs - Japanese Style,"Salmon - Atlantic, Skin On",Foam Dinner Plate


## Recommending Items to a New Customer

Suppose we get a new customer and on their first visit, they purchase the following items and quantities.

In [292]:
new_customer = {'Cookies - Assorted':3,
                'Flavouring - Orange':3,
                'Fenngreek Seed':1,
                'Wine - White Cab Sauv.on':1,
                'Bandage - Flexible Neon':3,
                'Oil - Shortening - All - Purpose':2,
                'Beef - Montreal Smoked Brisket':4,
                'French Pastry - Mini Chocolate':4,
                'Snapple Lemon Tea':5,
                'Pepper - White, Ground':2,
                'Spinach - Baby':5,
                'Sole - Dover, Whole, Fresh':4}

### Step 7: Recommend 5 products to this new customer using a user similarity approach.

### Step 8: Recommend 5 products to this new customer using a item similarity approach.

In [298]:
def recomend(customer):
    df = []
    prod = [x for x in customer]
    #Recomendación de 5 productos similares a los productos comprados
    top_related_product = [prod_dist[product].sort_values(ascending=False)[1:].head().index[i] for i in range(4) for product in prod]
    #Eliminamos los productos que ya ha comprado el cliente
    recom = [x for x in top_related_product if not x in prod]
    #Filtramos los 5 productos más recomendados en función de los productos comprados
    srecom = set(recom)
    sprod = {}
    for x in srecom:
        sprod[x] = recom.count(x)
    sprod = pd.DataFrame(sprod.items())
    sprod = sprod.sort_values(by=[1], ascending=False).head()
    #Agregamos los valores de cada cliente a una lista con todos los clientes
    df.append([x for x in sprod[0]])    
    #Creo un dataframe con los productos recomendados para cada cliente
    df = pd.DataFrame(df, columns=["Producto 1","Producto 2","Producto 3","Producto 4","Producto 5"])
    return df

In [299]:
recomend(new_customer)

,Producto 1,Producto 2,Producto 3,Producto 4,Producto 5
0,Initation Crab Meat,Squid - Tubes / Tenticles 10/20,Loquat,Tomatoes Tear Drop,Muffin Chocolate Individual Wrap
